In [424]:
# All general imports 
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import plot_model
from sklearn.metrics import f1_score, precision_score, recall_score

In [425]:
# All imports for visualizing the CNN model
from IPython.display import Image, display, HTML
from IPython.display import display as CoreDisplay
from IPython.display import Image
from IPython.display import display
from IPython.display import HTML
from IPython.display import display, Image
from IPython.display import display_html
from IPython.display import clear_output
from IPython.display import Markdown
from IPython.display import SVG
from IPython.display import IFrame
from ipywidgets import HBox, VBox
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pydot

In [426]:
# Load data from CSV file
data = pd.read_csv('mel_spec_dataset.csv')

# Split data into features and labels
X = data['file']
y = data['label']

# Define number of folds
k = 5

# Create k-fold cross-validator
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Initialize list to store evaluation results
scores = []

In [427]:
# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
custom_lr = 0.001
optimizer = Adam(learning_rate=custom_lr)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Visualizes the model using plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
model_image = widgets.Image(value=open('model.png', 'rb').read())

# Create a graph
graph = pydot.Dot(graph_type='digraph')

# Add nodes to the graph
for layer in model.layers:
    if isinstance(layer, Conv2D):
        # Convolutional layer
        node = pydot.Node(layer.name, label=f"Conv2D\n{layer.kernel_size}\n{layer.filters}", shape='rect')
    elif isinstance(layer, MaxPooling2D):
        # Max pooling layer
        node = pydot.Node(layer.name, label='MaxPooling2D', shape='oval')
    elif isinstance(layer, Dense):
        # Fully connected layer
        node = pydot.Node(layer.name, label=f"Dense\n{layer.units}", shape='diamond')
    else:
        # Other layer types
        node = pydot.Node(layer.name, label=layer.__class__.__name__)
    graph.add_node(node)

# Add edges to the graph
for i, layer in enumerate(model.layers):
    if i > 0:
        prev_layer = model.layers[i-1]
        graph.add_edge(pydot.Edge(prev_layer.name, layer.name))

# Save the graph to a file
graph.write_png('model_graph.png')
graph_image = widgets.Image(value=open('model_graph.png', 'rb').read())

# Display the images side by side
display(widgets.HBox([model_image, VBox([graph_image], layout={'align_items': 'center', 'padding': '150px', 'width' : '25%'})]))

In [428]:
# Loop over the folds
for i, (train_idx, test_idx) in enumerate(cv.split(X, y)):
    print(f"Fold {i+1}/{k}")
    
    # Split data into training and testing sets for this fold
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]

    train_data = pd.concat([X_train, y_train], axis=1)
    test_data = pd.concat([X_test, y_test], axis=1)
    
    # Create image data generators
    train_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    # Define batch size
    batch_size = 32
    
    # Generate training and testing data
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_data,
        directory='images_complete_proto_final',
        x_col='file',
        y_col='label',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='raw'
    )

    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_data,
        directory='images_complete_proto_final',
        x_col='file',
        y_col='label',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='raw'
    )

    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=10,
        #validation_data=test_generator,
        #validation_steps=test_generator.samples // batch_size
    )
    
    # Evaluate the model on the testing set
    y_pred = model.predict(test_generator)
    y_pred = np.where(y_pred > 0.5, 1, 0)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    score = model.evaluate(test_generator)
    scores.append([score[0], score[1], precision, recall, f1])

Fold 1/5
Found 84 validated image filenames.
Found 22 validated image filenames.
Epoch 1/10
2/2 [==============================] - 1s 260ms/step - loss: 0.9078 - accuracy: 0.3654
Epoch 2/10
2/2 [==============================] - 1s 371ms/step - loss: 0.6751 - accuracy: 0.5625
Epoch 3/10
2/2 [==============================] - 1s 227ms/step - loss: 0.7269 - accuracy: 0.4038
Epoch 4/10
2/2 [==============================] - 1s 421ms/step - loss: 0.6872 - accuracy: 0.6094
Epoch 5/10


KeyboardInterrupt: 

In [ ]:

# Compute the average scores across all folds
avg_score = np.mean(scores, axis=0)

print("Average results:")
print()
print(f"Loss = {avg_score[0]:.4f}")
print(f"Accuracy = {avg_score[1]:.4f}")
print(f"Precision = {avg_score[2]:.4f}")
print(f"Recall = {avg_score[3]:.4f}")
print(f"F1 = {avg_score[4]:.4f}")

Average results:

Loss = 0.1874
Accuracy = 0.9351
Precision = 0.5429
Recall = 0.5000
F1 = 0.5176
